In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
eps = np.finfo(float).eps

In [8]:
df=pd.read_csv("train.csv")

In [9]:
mid_point={} # dictionary containing mid 
class nodestructure:
    def __init__(self, value, pos=0, neg=0, left=None, right=None):
        self.value=value
        self.left=left
        self.right=right
        self.positive=pos
        self.negative=neg

In [6]:
left=df['left']
df=df.drop(columns=['left'])
df = pd.concat([df,pd.get_dummies(df['sales'], prefix='sales')],axis=1)
df = pd.concat([df,pd.get_dummies(df['salary'], prefix='salary')],axis=1)
df.drop(['sales','salary'],axis=1, inplace=True)
df=df.join(left)

KeyError: 'sales'

In [70]:
numericalfeatures = ['number_project','last_evaluation','satisfaction_level','average_montly_hours','time_spend_company']

In [71]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [72]:
def find_entropy(df):
    target = df.keys()[-1]   
    entropy = 0
    values = df[target].unique()
    for value in values:
        fraction = df[target].value_counts()[value]/(len(df[target])+eps)
        entropy += -fraction*np.log2(fraction+eps)
    return entropy  

In [73]:
def find_entropy_attribute(df,attribute):
    target = df.keys()[-1]   
    #Will return the unique values present in the target column 'left'
    target_variables = df[target].unique() 
    variables = df[attribute].unique()
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[target] ==target_variable])
            den = len(df[attribute][df[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*np.log2(fraction+eps)
        fraction2 =(den)/len(df)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

In [74]:
def get_subtable1(X_train,node,value):
    return X_train[X_train[node] < value].reset_index(drop=True)

In [75]:
def get_subtable2(X_train,node,value):
    return X_train[X_train[node] >= value].reset_index(drop=True)

In [76]:
def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:       
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

In [77]:
def get_subtable(df,node,value):
    return df[df[node] == value].reset_index(drop=True)

In [78]:
for feature in numericalfeatures:
    IGMax=0
    midmax=0
    prev=None
    train.sort_values(feature,inplace=True)
    for index,row in train.iterrows():
        if prev is not None:
            if prev['left']!=row['left']:
                mid=(float(prev[feature])+float(row[feature]))/2
                subtable1=get_subtable1(train,feature,mid)
                subtable2=get_subtable2(train,feature,mid)
                firstentropy=0
                secondentropy=0
                if subtable1.empty:
                    firstentropy=0
                else:
                    frac1=float(len(subtable1))/len(train)
                    firstentropy = frac1*find_entropy(subtable1)
                if subtable2.empty:
                    secondentropy=0
                else:
                    frac2=float(len(subtable2))/len(train)
                    secondentropy = frac2*find_entropy(subtable2)
                
                entropy1=find_entropy(train)
                entropy2=firstentropy+secondentropy
                ig=abs(entropy1)-abs(entropy2)
                if ig > IGMax:
                    IGMax=ig
                    midmax=mid

        prev=row
    mid_point[feature]=midmax
    for index,row in train.iterrows():
        if float(row[feature])>=midmax:
            train.at[index,feature]=1
        else:
            train.at[index,feature]=0

/home/vagupta/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [79]:
def buildtree(train):
    if len(train.columns)==1:
        return None
    output=train.keys()[-1]
    node= find_winner(train)
    root=nodestructure(node)
    outputValue,counts = np.unique(train[output],return_counts=True)                        
    if len(counts)<=1:
        if outputValue[0]==0:
            root.negative=counts[0]
        else:
            root.positive=counts[0]
    else:
        if outputValue[1]==0:
            root.negative=counts[1]
        else:
            root.positive=counts[1]
        if outputValue[0]==0:
            root.negative=counts[0]
        else:
            root.positive=counts[0]
        attValue=train[node].unique()
        for val in attValue:
            subdataframe = get_subtable(train,node,val)                     
            subdataframe=subdataframe.drop([node],axis=1)
            outputValue,counts = np.unique(subdataframe[output],return_counts=True)                        
            if len(counts)<=1:
                pass
            else:
                if val == 1:
                    root.right = buildtree(subdataframe)
                else:
                    root.left = buildtree(subdataframe)

    return root

In [1]:
rootNode=buildtree(train)
# print rootNode.value
# print rootNode.left.value
# print rootNode.right.value

NameError: name 'buildtree' is not defined

In [81]:
def testing(X_test,root):
    
    if root.left == None and root.right == None:
        if root.positive>root.negative:
            y_pred.append(1)
        else:
            y_pred.append(0)
        return
    
    j = root.value
    if j in X_test:
        if X_test[j] == 1:
            if root.right==None:
                if root.positive>root.negative:
                    y_pred.append(1)
                else:
                    y_pred.append(0)
            else:
                testing(X_test,root.right)
        else:
            if root.left==None:
                if root.positive>root.negative:
                    y_pred.append(1)
                else:
                    y_pred.append(0)
            else:
                testing(X_test,root.left)
    else:
        if root.left==None:
            if root.positive>root.negative:
                y_pred.append(1)
            else:
                y_pred.append(0)
        else:
            testing(X_test,root.left)

In [82]:
def convert_to_binary(X_test):
    for feature in numericalfeatures:
        for index,row in X_test.iterrows():
            if float(row[feature]) >= mid_point[feature]:
                X_test.at[index,feature]=1
            else:
                X_test.at[index,feature]=0


In [83]:
y_pred=[]



Y_test=test['left']
X_test=test.drop(['left'],axis=1)
# X_test
convert_to_binary(X_test)

for index,row in X_test.iterrows():
    testing(row,rootNode)
    
# print y_pred

print confusion_matrix(Y_test,y_pred)
print classification_report(Y_test,y_pred)
print accuracy_score(Y_test, y_pred)

[[1696   42]
 [ 236  315]]
              precision    recall  f1-score   support

           0       0.88      0.98      0.92      1738
           1       0.88      0.57      0.69       551

   micro avg       0.88      0.88      0.88      2289
   macro avg       0.88      0.77      0.81      2289
weighted avg       0.88      0.88      0.87      2289

0.8785495849716033


In [84]:
def predictentropy(rootNode,mid_point,test):
    def testing(X_test,root):
        if root.left == None and root.right == None:
            if root.positive>root.negative:
                y_pred.append(1)
            else:
                y_pred.append(0)
            return

        j = root.value
        if j in X_test:
            if X_test[j] == 1:
                if root.right==None:
                    if root.positive>root.negative:
                        y_pred.append(1)
                    else:
                        y_pred.append(0)
                else:
                    testing(X_test,root.right)
            else:
                if root.left==None:
                    if root.positive>root.negative:
                        y_pred.append(1)
                    else:
                        y_pred.append(0)
                else:
                    testing(X_test,root.left)
        else:
            if root.left==None:
                if root.positive>root.negative:
                    y_pred.append(1)
                else:
                    y_pred.append(0)
            else:
                testing(X_test,root.left)
    
    def convert_to_binary(X_test):
        for feature in numericalfeatures:
            for index,row in X_test.iterrows():
                if float(row[feature]) >= mid_point[feature]:
                    X_test.at[index,feature]=1
                else:
                    X_test.at[index,feature]=0

    y_pred=[]
    Y_test=test['left']
    X_test=test.drop(['left'],axis=1)
    convert_to_binary(X_test)
    for index,row in X_test.iterrows():
        testing(row,rootNode)
    print confusion_matrix(Y_test,y_pred)
    print classification_report(Y_test,y_pred)
    print accuracy_score(Y_test, y_pred)

In [85]:
X_test1 = pd.read_csv("sample_test.csv")
X_test1 = pd.concat([X_test1,pd.get_dummies(X_test1['sales'], prefix='sales')],axis=1)
X_test1 = pd.concat([X_test1,pd.get_dummies(X_test1['salary'], prefix='salary')],axis=1)
X_test1.drop(['sales','salary'],axis=1, inplace=True)

predictentropy(rootNode,mid_point,X_test1)

[[1]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

1.0


In [86]:
df = pd.read_csv("train.csv")
X = df.drop(['left'],axis=1)
Y = df['left']
X = pd.concat([X,pd.get_dummies(X['sales'], prefix='sales')],axis=1)
X = pd.concat([X,pd.get_dummies(X['salary'], prefix='salary')],axis=1)
X.drop(['sales','salary'],axis=1, inplace=True)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2)
model = DecisionTreeClassifier()
model.fit(X_train,Y_train)
Y_predict=model.predict(X_test)
print confusion_matrix(Y_test,Y_predict)
print classification_report(Y_test,Y_predict)
print accuracy_score(Y_test, Y_predict)

[[1666   45]
 [  25  512]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1711
           1       0.92      0.95      0.94       537

   micro avg       0.97      0.97      0.97      2248
   macro avg       0.95      0.96      0.96      2248
weighted avg       0.97      0.97      0.97      2248

0.9688612099644128
